Get the desired cell from Allen DB

In [ ]:
download_cell = input("True or False: Do you want to download the cell? ")
if download_cell not in [True, False]:
    ValueError("Enter bool: True or False")
if download_cell == "True":
    print("Downloading the cell")
else:
    print("Not downloading the cell")

In [1]:
# !pip install allensdk

In [2]:
cell_id = 477127614
cell_dir = f'Allen/Cell_{cell_id}'

In [3]:
from allensdk.api.queries.biophysical_api import BiophysicalApi
from allensdk.model.biophys_sim.config import Config
from allensdk.model.biophysical.utils import Utils
import re
import json
import numpy as np
import subprocess
import sys
import os


In [4]:
if not os.path.exists(cell_dir):
    os.mkdir(cell_dir)

In [5]:
os.chdir(cell_dir)

In [ ]:
if download_cell:
    bp = BiophysicalApi()
    query = bp.get_neuronal_models(cell_id)
    query

[{'id': 497232641,
  'name': 'Biophysical - all active_Slc17a6-IRES-Cre;Ai14-190263.04.01.01',
  'neuron_reconstruction_id': 491392956,
  'neuronal_model_template_id': 491455321,
  'specimen_id': 477127614},
 {'id': 478809991,
  'name': 'Biophysical - perisomatic_Slc17a6-IRES-Cre;Ai14-190263.04.01.01',
  'neuron_reconstruction_id': 491392956,
  'neuronal_model_template_id': 329230710,
  'specimen_id': 477127614}]

In [ ]:
if download_cell:
    q_idx = int(input(f"Choose which query you would like to use (0-{len(query)-1}): {[cell['name'] for cell in query]}"))
    if q_idx < 0 or q_idx >= len(query):
        ValueError("Invalid query index: {q_idx}")

In [ ]:
if download_cell:
    bp.cache_stimulus = False # Change to False to not download the large stimulus NWB file
    bp.cache_data(query[q_idx]['id']) # 'id'

2025-03-18 10:43:10,463 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/706381807
2025-03-18 10:43:10,844 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/491392958
2025-03-18 10:43:11,098 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/706381810
2025-03-18 10:43:11,240 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/496607496
2025-03-18 10:43:11,441 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/395337019
2025-03-18 10:43:11,588 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/395337225
2025-03-18 10:43:11,727 allensdk.api.api.retrieve_fi

In [ ]:
# add synapse modfiles
%cp ../../modfiles/synapses/* modfiles/

Build the Cell

In [ ]:
def update_missing_passive_values(utils, user_specs_dict):
  # update missing properties to user_specs_dict if they're not already in the allen specifications
  if "e_pas" not in utils.description.data["passive"][0].keys():
    utils.description.data["passive"][0]["e_pas"] = user_specs_dict["e_pas"]
    print(f"e_pas not in Allen specs, updating to {user_specs_dict['e_pas']}")
  
  if "cm" not in utils.description.data["passive"][0].keys():
    utils.description.data["passive"][0]["cm"] = user_specs_dict["cm"]
    print(f"cm not in Allen specs, updating to {user_specs_dict['cm']}")

  if "ra" not in utils.description.data["passive"][0].keys():
    utils.description.data["passive"][0]["ra"] = user_specs_dict["ra"]
    print(f"ra not in Allen specs, updating to {user_specs_dict['ra']}")

  return utils

def load_dictionary_from_json(file_path):
  """Loads a dictionary from a JSON file.

  Args:
    file_path: The path to the JSON file.

  Returns:
    A dictionary loaded from the JSON file.

  Raises:
    ValueError: If the JSON file is invalid or cannot be parsed.
    FileNotFoundError: If the specified file does not exist.
  """
  try:
    with open(file_path, 'r') as file:
      try:
        data = json.load(file)
        if not isinstance(data, type({})):
          raise ValueError("JSON file does not contain a dictionary.")
        print(f"loaded user_specs from {file_path}")
        return data
      except json.JSONDecodeError as e:
        raise ValueError(f"Invalid JSON format: {e}") from e
  except FileNotFoundError:
    raise FileNotFoundError(f"File not found: {file_path}")

In [26]:
# compile the downloaded modfiles
if not os.path.exists("x86_64"):
    print("compiling modfiles")
    subprocess.run("nrnivmodl modfiles", shell=True, check=True)
else:
    print("modfiles already compiled. skipping")

# Create the h object
description = Config().load('manifest.json')
utils = Utils(description)
h = utils.h

# convert 'values' from string to float
for dict in utils.description.data['genome']:
    for key,item in dict.items():
        if key == 'value':
            dict[key] = float(item)

# load user specifications
user_specs_dict = load_dictionary_from_json("../user_specifications.json")

if user_specs_dict:
    utils = update_missing_passive_values(utils, user_specs_dict)

# read morphology
manifest = description.manifest
morphology_path = description.manifest.get_path('MORPHOLOGY')
utils.generate_morphology(morphology_path.encode('ascii', 'ignore'))

# build the cell. Its parts will be assigned to the h object
utils.load_cell_parameters()

compiling modfiles


/home/drfrbc/miniconda3/envs/sim_env/bin/nrnivmodl:10: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import working_set


/home/drfrbc/Neural-Modeling/Allen/Cell_477127614
Mod files: "modfiles/modfiles/AMPA_NMDA.mod" "modfiles/modfiles/AMPA_NMDA_STP_LTP.mod" "modfiles/modfiles/AMPA_NMDA_STP.mod" "modfiles/modfiles/CaDynamics.mod" "modfiles/modfiles/Ca_HVA.mod" "modfiles/modfiles/Ca_LVA.mod" "modfiles/modfiles/GABA_AB.mod" "modfiles/modfiles/GABA_AB_STP.mod" "modfiles/modfiles/Ih.mod" "modfiles/modfiles/Im.mod" "modfiles/modfiles/Im_v2.mod" "modfiles/modfiles/int2pyr.mod" "modfiles/modfiles/Kd.mod" "modfiles/modfiles/K_P.mod" "modfiles/modfiles/K_T.mod" "modfiles/modfiles/Kv2like.mod" "modfiles/modfiles/Kv3_1.mod" "modfiles/modfiles/Nap.mod" "modfiles/modfiles/NaTa.mod" "modfiles/modfiles/NaTs.mod" "modfiles/modfiles/NaV.mod" "modfiles/modfiles/pyr2pyr.mod" "modfiles/modfiles/pyr2pyrSUM.mod" "modfiles/modfiles/SK.mod" "modfiles/modfiles/vecevent.mod"

Creating 'x86_64' directory for .o files.

 -> Compiling mod_func.cpp
 -> NMODL ../modfiles/AMPA_NMDA.mod
 -> NMODL ../modfiles/AMPA_NMDA_STP.mod
 -> NMODL .

Translating AMPA_NMDA_STP.mod into /home/drfrbc/Neural-Modeling/Allen/Cell_477127614/x86_64/AMPA_NMDA_STP.c
Translating AMPA_NMDA.mod into /home/drfrbc/Neural-Modeling/Allen/Cell_477127614/x86_64/AMPA_NMDA.c
Translating AMPA_NMDA_STP_LTP.mod into /home/drfrbc/Neural-Modeling/Allen/Cell_477127614/x86_64/AMPA_NMDA_STP_LTP.c
Thread Safe
Thread Safe
Thread Safe
Translating CaDynamics.mod into /home/drfrbc/Neural-Modeling/Allen/Cell_477127614/x86_64/CaDynamics.c
Translating Ca_HVA.mod into /home/drfrbc/Neural-Modeling/Allen/Cell_477127614/x86_64/Ca_HVA.c
Thread Safe
Thread Safe
Translating Ca_LVA.mod into /home/drfrbc/Neural-Modeling/Allen/Cell_477127614/x86_64/Ca_LVA.c
Thread Safe
Translating GABA_AB.mod into /home/drfrbc/Neural-Modeling/Allen/Cell_477127614/x86_64/GABA_AB.c
Translating GABA_AB_STP.mod into /home/drfrbc/Neural-Modeling/Allen/Cell_477127614/x86_64/GABA_AB_STP.c
Thread Safe
Thread Safe
Translating Ih.mod into /home/drfrbc/Neural-Modeling/Allen/Cell_477127614/x86_64/Ih.c
Thre

 -> Compiling Ca_HVA.c
 -> Compiling Ca_LVA.c
 -> Compiling GABA_AB.c
 -> Compiling GABA_AB_STP.c
 -> Compiling Ih.c
 -> Compiling Im.c
 -> Compiling Im_v2.c
 -> Compiling int2pyr.c
 -> Compiling Kd.c
 -> Compiling K_P.c
 -> Compiling K_T.c
 -> Compiling Kv2like.c
 -> Compiling Kv3_1.c
 -> Compiling Nap.c
 -> Compiling NaTa.c
 -> Compiling NaTs.c
 -> Compiling NaV.c
 -> Compiling pyr2pyr.c
 -> Compiling pyr2pyrSUM.c
 -> Compiling SK.c
 -> Compiling vecevent.c
 => LINKING shared library ./libnrnmech.so
 => LINKING executable ./special LDFLAGS are:    -pthread
Successfully created x86_64/special


NEURON: NaV is not a MECHANISM
 near line 1
 forsec "axon" { insert NaV }
                          ^
NEURON: K_T is not a MECHANISM
 near line 1
 forsec "axon" { insert K_T }
                          ^
NEURON: Kd is not a MECHANISM
 near line 1
 forsec "axon" { insert Kd }
                         ^
NEURON: Kv2like is not a MECHANISM
 near line 1
 forsec "axon" { insert Kv2like }
                              ^
NEURON: Kv3_1 is not a MECHANISM
 near line 1
 forsec "axon" { insert Kv3_1 }
                            ^
NEURON: SK is not a MECHANISM
 near line 1
 forsec "axon" { insert SK }
                         ^
NEURON: Ca_HVA is not a MECHANISM
 near line 1
 forsec "axon" { insert Ca_HVA }
                             ^
NEURON: Ca_LVA is not a MECHANISM
 near line 1
 forsec "axon" { insert Ca_LVA }
                             ^
NEURON: CaDynamics is not a MECHANISM
 near line 1
 forsec "axon" { insert CaDynamics }
                                 ^
NEURON: CaDynamics is not a MEC

In [37]:
class SkeletonCell():
    def __init__(self, h):
        self.soma = list(h.soma)
        self.all = [sec for sec in h.allsec()]
        self.apic = list(h.apic)
        self.dend = list(h.dend)
        self.axon = list(h.axon)

In [38]:
skeleton_cell = SkeletonCell(utils.h)

In [39]:
skeleton_cell.soma

[soma[0]]